In [32]:
# set up
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# supress the warnings
import warnings
warnings.filterwarnings('ignore')

In [33]:
# load the data
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
training_iterator = data_generator.flow_from_directory(
    "Covid19-dataset/train",
    class_mode="categorical",
    color_mode="grayscale",
    target_size=(256, 256),
    batch_size=32,
    shuffle=True,
)
test_iterator = data_generator.flow_from_directory(
    "Covid19-dataset/test",
    class_mode="categorical",
    color_mode="grayscale",
    target_size=(256, 256),
    batch_size=32,
    shuffle=True,
)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [108]:
# design a classification neural network model
def cnn_model(training_data):
    # create the model
    model_cnn = Sequential()

    # add some hidden layers
    model_cnn.add(Conv2D(32, 3, strides=2, activation='relu', input_shape=(256, 256, 1)))
    model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
    model_cnn.add(Dropout(0.1))
    
    model_cnn.add(Conv2D(64, 3, strides=2, activation='relu'))
    model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
    model_cnn.add(Dropout(0.1))
    
    # flatten the data
    model_cnn.add(Flatten())
    
    # add a dense layer
    model_cnn.add(Dense(512, activation='relu'))
    model_cnn.add(Dropout(0.1))
    
    
    # create an output layer
    model_cnn.add(Dense(3, activation='softmax'))
    
    # compile the model
    model_cnn.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )
    
    model_cnn.summary()
    return model_cnn

In [109]:
# create an early stopping callback
es = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [110]:
# fit the model
model = cnn_model(training_iterator)
history = model.fit(
    training_iterator,
    steps_per_epoch = training_iterator.samples//32,
    validation_data = test_iterator,
    validation_steps = test_iterator.samples//32,
    epochs = 20,
    callbacks=[es]
)

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 127, 127, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 31, 31, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 14400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 512)            │     7,373,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,393,667 (28.20 MB)

 Trainable params: 7,393,667 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 195ms/step - accuracy: 0.3468 - loss: 2.0630 - val_accuracy: 0.5000 - val_loss: 1.1305
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3750 - loss: 1.2940 - val_accuracy: 0.5000 - val_loss: 0.9253
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 132ms/step - accuracy: 0.4862 - loss: 1.0700 - val_accuracy: 0.5469 - val_loss: 1.0602
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6875 - loss: 0.9980 - val_accuracy: 1.0000 - val_loss: 0.9885
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 132ms/step - accuracy: 0.6584 - loss: 0.9719 - val_accuracy: 0.5625 - val_loss: 0.9663
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5926 - loss: 0.9028 - val_accuracy: 0.5000 - val_loss: 0.9781
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 0.6482 - loss: 0.8231 - val_accuracy: 0.6094 - val_loss: 0.8398
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5938 - loss: 0.8614 - val_accuracy: 1.0000 - val_loss: 0.5522


In [140]:
  # evaluate the model
loss, acc = model.evaluate(test_iterator)
print(f"Loss: {loss:.4f}, Accuracy: {acc:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.7232 - loss: 0.7654 
Loss: 0.7436, Accuracy: 0.7121
